In [1]:
# shopee.py
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time

# Scraping Link Produk

## Fungsi Scraping Link Produk
Fungsi untuk scraping link produk yang sesuai dengan kata kunci

In [2]:

def scrapeLink(list_product, category):
    chrome_options = Options()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=2')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    # katakunci = input('Masukkan kata kunci : ')

    def search(katakunci):
        links = []
        print('mencari semua product dengan kata kunci ' + katakunci)
        url = 'https://shopee.co.id/search?keyword=' + katakunci
        try:
            driver.get(url)
            time.sleep(5)
            driver.execute_script('window.scrollTo(0, 1500);')
            time.sleep(5)
            driver.execute_script('window.scrollTo(0, 2500);')
            time.sleep(5)
            soup_a = BeautifulSoup(driver.page_source, 'html.parser')
            products = soup_a.find('div', class_='row shopee-search-item-result__items')
            for link in products.find_all('a'):
                links.append('https://shopee.co.id/' + str(link.get('href')))
                # print(link.get('href'))
        except TimeoutException:
            print('failed to get links with query ' )
        return links
    
    for katakunci in list_product:
        print('Scrape link ' + katakunci)
        product_urls = search(katakunci)
        dict_urls = {'url' : product_urls}
        df = pd.DataFrame(data = dict_urls)
        PATH = '../data/raw/'+ category +'/link_product'
        file_name = PATH + '/links_' + katakunci +'.csv'
        df.to_csv(file_name, index=False)


## Kategori dan Kata Kunci
Menginisiasi list kategori dan kata kunci

In [3]:
CATEGORY = ['fashion', 'food', 'electronic']

fashion = ['hoodie', 'hijab', 'celana', 'kemeja', 'gamis', 'sepatu', 'kaos', 'sandal', 'tas', 'jeans', 'jaket', 'sweater']
food = ['snack', 'daging', 'selai', 'gula', 'susu', 'mie', 'roti', 'bumbu', 'camilan', 'bahan_kue', 'minuman', 'bahan_pokok']
electronic = ['laptop', 'mouse', 'handphone', 'earphone', 'tv', 'mixer', 'setrika', 'mesin_cuci', 'penyedot_debu', 'kamera', 'speaker', 'lampu']


## Scraping Link Produk
Scraping link produk berdasarkan kata kunci

In [4]:
# for category in [fashion, food, electronic]:
#     scrapeLink(category)

# Scraping Ulasan Produk
Scraping ulasan produk dari link yang telah ada

In [5]:
from os import walk
import re
import timeit
import requests

## Fungsi Scraping Ulasan

In [6]:
def shopeeScraper (url):
    url = url
    r = re.search(r'i\.(\d+)\.(\d+)', url)
    shop_id, item_id = r[1], r[2]
    ratings_url = 'https://shopee.co.id/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data_scrape = []
    start = timeit.default_timer()
    runtime = 0
    offset = 0
    print('Scraping Process...')
    while True:   
        data = requests.get(ratings_url.format(shop_id=shop_id, item_id=item_id, offset=offset)).json()

        i = 1
        try :
            for i, rating in enumerate(data['data']['ratings'], 1):
                if rating['comment'] == '':
                    pass
                else:
                    data_scrape.append([rating['rating_star'], rating['comment']])
        except :
            pass

        if i % 20:
            break

        stop = timeit.default_timer()
        runtime+= (stop-start)
        print(runtime)
        offset += 20
    print('Scraping Done.')
    stop = timeit.default_timer()
    print('Time: ', stop - start) 
    df = pd.DataFrame(data_scrape, columns=['rating', 'reviews'])
    df = df.dropna(axis=0)
    return df

## List File Link
Membuat list file dari direktori link_product

In [7]:
LINK_PRODUCT = {'fashion' : [], 'food' : [], 'electronic' : []}

for ctg in CATEGORY:
    for (dirpath, dirnames, filenames) in walk('../data/raw/link_product/' + ctg):
        LINK_PRODUCT[ctg].extend(filenames)
        break


## Scraping Ulasan Produk
Scraping Ulasan Produk dari daftar link produk di folder link_product

In [8]:
dataReviews = pd.DataFrame({'rating' : [], 'reviews' : []})
ctg = 'electronic'
file = LINK_PRODUCT[ctg][11]
links = pd.read_csv('../data/raw/link_product/' + ctg + '/' + file)
for i, link in enumerate(links['url']):
    if i >= 25:
        print('Scraping ' + file + ' Link ke-' + str(i))
        reviews = shopeeScraper(link)
        frames = [dataReviews, reviews]
        result = pd.concat(frames).drop_duplicates().reset_index(drop=True)
        dataReviews = result
        if i % 2 == 0:
            dataReviews.to_csv('../data/raw/reviews_product/' + ctg + '/tv_v' + str(i/2) + '.csv', index=False)


Scraping links_tv.csv Link ke-25
Scraping Process...
0.42714950000000007
1.1254487000000002
2.0943875000000003
3.4226019000000005
5.015219500000001
6.880645000000001
9.061552100000002
11.581383300000002
14.395166600000003
17.485623400000005
20.853892600000005
24.563071400000005
28.558010900000006
32.822801600000005
37.373642600000004
42.216816400000006
47.330157500000006
52.72411890000001
58.38820360000001
64.3300147
70.74766640000001
77.44385200000002
84.41195090000002
91.64787590000002
99.20487680000002
107.02774490000002
115.10434010000002
123.44694190000001
132.1148335
141.06569480000002
150.27496290000002
159.74887030000002
169.46763850000002
179.56763120000002
189.93305410000002
200.57989540000003
211.49929290000003
222.82232220000003
234.41587270000002
246.27139680000002
258.37094
270.73343470000003
283.50659110000004
296.53218780000003
309.83276570000004
323.5280875
337.49515620000005
351.70433090000006
366.19097280000005
380.9255087000001
396.0958227000001
411.5183084000001
42

In [10]:
FASHION_LINK_MERGE = []
ctg = 'electronic' # Ganti dengan kategori data yang ingin digabungkan
for (dirpath, dirnames, filenames) in walk('../data/raw/reviews_product/' + ctg):
        FASHION_LINK_MERGE.extend(filenames) # Membuat list nama file dari direktori kategori
        break

merge_list, count_data = [], 0
for i in FASHION_LINK_MERGE:
    data = pd.read_csv('../data/raw/reviews_product/' + ctg + '/' + i)
    count_data += len(data)
    merge_list.append(data)
merge_result = pd.concat(merge_list).drop_duplicates().reset_index(drop=True)
merge_result.to_csv('../data/raw/' + ctg + '_reviews.csv', index=False)

print(count_data)
print(merge_result)

2323987
        rating                                            reviews
0          5.0  Sesuai ekspektasi banget sih ini. Warnanya can...
1          5.0  Alhamdulillah paket nya sudah sampai dgn aman,...
2          5.0  Paket sudah sampai, warna nya cantik banget he...
3          5.0  bagusss bangettt beli pas lagi promoo tapi ga ...
4          5.0  Produk ori miniso, barang bagus, nyampe dengan...
...        ...                                                ...
365688     5.0                           Mantaab...menggelegaarrr
365689     5.0     pesen jam 12, jam 4 udh nyampe\nmakasih seller
365690     5.0            Terimakasih atas pelayanan nya yg cepat
365691     5.0                                      Cepat makasih
365692     5.0                                     Kereeeen 👍👍👍👍👍

[365693 rows x 2 columns]
